# 快速入门 GPT-4 Vison

从历史上看，语言模型系统仅接受**文本**作为输入。但是单一的输入形式，限制了大模型的应用落地范围。

随着技术发展，OpenAI 开发的 GPT-4 Turbo with Vision（简称 GPT-4V）允许模型接收**图像**作为输入，并回答关于它们的问题。

📢注意，目前在 Assistants API 中使用 GPT-4 时还不支持图像输入。

## 使用 GPT-4V 识别线上图像（URL）

![image_sample](https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg)

In [1]:
from openai import OpenAI

client = OpenAI()

response = client.chat.completions.create(
  model="gpt-4-turbo",
  messages=[
    {
      "role": "user",
      "content": [
        {"type": "text", "text": "介绍下这幅图?"},
        {
          "type": "image_url",
          "image_url": {
            "url": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg",
          },
        },
      ],
    }
  ],
  max_tokens=300,
)

print(response.choices[0])

Choice(finish_reason='length', index=0, logprobs=None, message=ChatCompletionMessage(content='这幅图呈现了一个宁静美丽的自然景观。图中的主要特点是一条木制的栈道，它穿越在郁郁葱葱的草地上，引向远处看不到的终点。草地上覆盖着茂密的绿色植被，显示出生机勃勃的春夏季节。背景中天空呈现出澄清的蓝色，点缀着几朵轻盈的白云，增添了一种宁静和平和的氛围。\n\n整个场景光线充足，阳光和蓝天与绿色植被的对比令人感觉清新愉快。这样的环境可能是理想的徒步旅行地点，提供了亲近自然和放松心情的完美机会。整体上，这幅图传达了大自然的宁静与和谐，是观赏和体验自然美景的一种', role='assistant', function_call=None, tool_calls=None))


In [2]:
response.choices[0].message.content

'这幅图呈现了一个宁静美丽的自然景观。图中的主要特点是一条木制的栈道，它穿越在郁郁葱葱的草地上，引向远处看不到的终点。草地上覆盖着茂密的绿色植被，显示出生机勃勃的春夏季节。背景中天空呈现出澄清的蓝色，点缀着几朵轻盈的白云，增添了一种宁静和平和的氛围。\n\n整个场景光线充足，阳光和蓝天与绿色植被的对比令人感觉清新愉快。这样的环境可能是理想的徒步旅行地点，提供了亲近自然和放松心情的完美机会。整体上，这幅图传达了大自然的宁静与和谐，是观赏和体验自然美景的一种'

### 封装成一个函数 query_image_description

In [3]:
def query_image_description(url, prompt="介绍下这幅图?"):
    client = OpenAI()  # 初始化 OpenAI 客户端
    
    # 发送请求给 OpenAI 的聊天模型
    response = client.chat.completions.create(
        model="gpt-4-turbo",  # 指定使用的模型
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": url}},
                ],
            }
        ],
        max_tokens=300,
    )
    
    # 返回模型的响应
    return response.choices[0].message.content


### 调用函数测试

![meme_0](https://p6.itc.cn/q_70/images03/20200602/0c267a0d3d814c9783659eb956969ba1.jpeg)

In [4]:
image_url = "https://p6.itc.cn/q_70/images03/20200602/0c267a0d3d814c9783659eb956969ba1.jpeg"
content = query_image_description(image_url)
print(content)

这幅图是一种幽默风格的搞笑比较。图中展示了两种不同状态的狗。左边是一只被幻想成拥有非常发达肌肉、类似人类健美运动员身体的柴犬，配文“16岁的我，工作后的我”，表达了年轻时充满活力和梦想的状态。右边的柴犬则显得普通、略显憔悴，配有文字“好累好困、好想摸鱼、看到鸡腿就开心，不属于我、我也有小肚腩、肉肉的很可爱”，描绘了工作后可能感到疲惫、减少锻炼而略显发福的现实状态。

这种对比通常用来幽默地表达人们对于理想与现实之间差距的感慨，展示了许多人对于年轻时的憧


### 使用 GPT-4V 识别本地图像文件（Base64编码）


In [1]:
from openai import OpenAI
import base64
import requests
import json

client = OpenAI()  # 初始化 OpenAI 客户端

def query_base64_image_description(image_path, prompt="解释下图里的内容？", max_tokens=1000):

    # 实现 Base64 编码
    def encode_image(path):
        with open(path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')

    # 获取图像的 Base64 编码字符串
    base64_image = encode_image(image_path)

    # 构造请求的 HTTP Header
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {client.api_key}"
    }

    # 构造请求的负载
    payload = {
        "model": "gpt-4-turbo",
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}}
                ]
            }
        ],
        "max_tokens": max_tokens
    }

    # 发送 HTTP 请求
    response = requests.post("https://api.xiaoai.plus/v1/chat/completions", headers=headers, json=payload)

    # 检查响应并提取所需的 content 字段
    if response.status_code == 200:
        response_data = response.json()
        content = response_data['choices'][0]['message']['content']
        return content
    else:
        return f"Error: {response.status_code}, {response.text}"

#### 使用 Assistants API生成的 GDP 40年对比曲线图

![gdp_data](./images/gdp_1980_2020.jpg)

In [2]:
content = query_base64_image_description("./images/gdp_1980_2020.jpg")
print(content)

这张图表显示了从1980年到2020年美国、中国、日本和德国的国内生产总值（GDP）比较。GDP是以万亿美元为单位度量的。从图表中可以看出以下几点：

1. **美国（蓝线）**：美国的GDP在这四十年里持续上升，从大约3万亿美元增长到接近22万亿美元。

2. **中国（红线）**：中国的GDP显示了显著的增长，特别是从2000年以后，增长速度加快，从不足1万亿美元跃升到超过14万亿美元。

3. **日本（紫线）**：日本的GDP在1990年代初达到高峰，之后有所波动但总体趋势较为平稳，在5万亿到6万亿美元之间。

4. **德国（绿线）**：德国的GDP也呈现增长趋势，尽管增幅不如中国那么剧烈，从1万亿美元左右增长至约4万亿美元。

总的来说，这张图表反映了不同国家经济增长的轨迹和速度，其中中国的快速发展尤为引人注目。


#### 使用 GPT-4V 识别手写体笔记

![](./images/handwriting_0.jpg)

In [3]:
content = query_base64_image_description("./images/handwriting_0.jpg")
print(content)

这张照片显示的是一本笔记本上记录的关于自然语言处理（NLP）的技术内容。主要是讨论了不同类型的模型调整方法，特别是用于处理大型语言模型的技术。

1. **Prompt Tuning（提示调整）**: 提示调整是一种用于微调预训练语言模型的技术。它通过增加少量可训练的嵌入（Token Embeddings）来指导模型在特定任务上的表现。在笔记中提到了使用这种技术对于小模型（Small Model）的适用性。

2. **Prefix Tuning（前缀调整）**: 这是另一种调整方法，其中前缀代表添加在输入序列前的可训练参数。这些前缀的目的是引导模型生成特定的输出。

3. **LoRA（低秩适应）**: LoRA是一种新的模型参数化技术，它通过低秩矩阵来调整模型中的权重，而不是替换原有的权重。这里提到，通过这种技术可以实现参数数量的有效减少，从而降低存储和计算成本。

4. **Q-LoRA**: 这似乎是基于LoRA技术的一种变体，具体细节在笔记中并未详尽说明，但可以推测其为一种优化查询（Query）的技术。

此外，笔记本上还记录了一些数据大小的信息，如"LAMA - 65GB 小LoRA - 78GB"，这可能是提到使用不同技术后模型的大小。

从上述内容可以看出，笔记主要在讨论和记录一些高级的语言模型调整技术，目的是为了提高模型的性能和效率。


#### 在 Jupyter 标准输出中渲染 Markdown 格式内容

In [4]:
from IPython.display import display, Markdown

# 使用 display 和 Markdown 函数显示 Markdown 内容
display(Markdown(content))

这张图片显示的是一页用中文手写的笔记，上面列有几个带有年份和编号的项目。从上到下，文字的大意如下：

1. 2021年
   - 1.制定计划
   - 2.坚持原则
   - 3.注重结果
2. 2021年
   - 1.决策迅速
   - 2.态度端正
   - 3.关注细节

这看起来像是某人为自己设定的年度目标或原则，可能用于个人发展或工作管理。每个项目前都标有一个数字，可能表示优先级或是执行顺序。

![](./images/handwriting_1.jpg)

In [ ]:
content = query_base64_image_description("./images/handwriting_1.jpg")
display(Markdown(content))

In [8]:
content = query_base64_image_description("./images/test.jpg")
display(Markdown(content))

这张图片展示的是一张笔记本的一页，其中记录了两组列表。每个列表标题上都标注了“2024年的计划”，下面列出了四个项目。这些可以理解为个人或工作计划，每个都针对未来的目标或任务。

第一个列表：
1. 继续学习
2. 继续努力
3. 继续深造
4. 继续坚持

第二个列表：
1. 减压
2. 慢生活
3. 释放
4. 物超所值

这些条目可能代表个人的学术或职业目标以及努力改善生活质量的愿望。第一个列表较多强调持续性和努力，涉及学习和个人成长。第二个列表则偏重于生活质量的提升，比如通过减压、享受慢生活等方式达成更平衡的生活状态。

## Homework: 


### #1

使用 GPT-4V 识别带有手写体文字的本地图像文件，分享结果。

### #2

整合 `query_base64_image_description` 函数和 Markdown 格式渲染方法，使得输出结果更易阅读。